## Importing Dependencies

In [1]:
import requests
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import ccxt
from tqdm.notebook import tqdm
import pandas as pd
import concurrent.futures

## Extracting trading pairs with highest market cap on Binance

In [2]:
url = 'https://coinmarketcap.com/exchanges/binance/'
response = requests.get(url)
content = response.text

In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
market_data_html_class = 'sc-7b3ac367-3 NFswn cmc-table'
table = soup.find(class_=market_data_html_class)

In [5]:
column_index = 2
trading_pairs = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > column_index:
        trading_pairs.append(cells[column_index].text)

for i in range(10):
    print(trading_pairs[i])

BTC/FDUSD
BTC/USDT
ETH/FDUSD
ETH/USDT
USDC/USDT
SOL/USDT
SOL/FDUSD
XRP/USDT
FDUSD/USDT
SUI/USDT


## Extracting OHLCV data for each trading pair

In [6]:
binance = ccxt.binance()

In [7]:
def fetch_ohlcv(trading_pair, interval, start_ts):
    from_ts = binance.parse8601(start_ts)
    ohlcv = binance.fetch_ohlcv(trading_pair, interval, since=from_ts, limit=1000)
    
    while True:
        from_ts = ohlcv[-1][0] + 1
        new_ohlcv = binance.fetch_ohlcv(trading_pair, interval, since=from_ts, limit=1000)
        ohlcv.extend(new_ohlcv)
        if len(new_ohlcv) != 1000:
            break

    data_dict = {trading_pair: ohlcv}
    return data_dict

In [8]:
start_ts = '2024-09-03 18:00:00'
interval = '30m'
full_ohlcv = {}
    
for trading_pair in tqdm(trading_pairs, desc='Fetching OHLCV Data', unit='Trading Pair'):
    ohlcv = fetch_ohlcv(trading_pair, interval, start_ts)
    full_ohlcv.update(ohlcv)

Fetching OHLCV Data:   0%|          | 0/50 [00:00<?, ?Trading Pair/s]

In [9]:
data_for_df = []

for trading_pair, ohlcv in full_ohlcv.items():
    for entry in ohlcv:
        data_for_df.append([entry[0], trading_pair] + entry[1:])

In [10]:
df = pd.DataFrame(data_for_df, columns=['Timestamp', 'Trading Pair', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.set_index('Timestamp', inplace=True)
df

,Trading Pair,Open,High,Low,Close,Volume
Timestamp,,,,,,
2024-09-03 18:00:00,BTC/FDUSD,57787.8500,58034.6300,57665.9300,57968.2300,8.237867e+02
2024-09-03 18:30:00,BTC/FDUSD,57971.0700,58141.4900,57953.5300,58003.1500,6.733895e+02
2024-09-03 19:00:00,BTC/FDUSD,58000.3600,58350.0000,57978.8600,58099.4200,6.882739e+02
2024-09-03 19:30:00,BTC/FDUSD,58099.5300,58202.0000,57936.2000,58138.1900,8.848367e+02
2024-09-03 20:00:00,BTC/FDUSD,58141.8800,58342.0000,58133.8500,58241.0700,4.732882e+02
...,...,...,...,...,...,...
2024-10-03 16:00:00,ARB/USDT,0.5264,0.5274,0.5180,0.5183,1.847456e+06
2024-10-03 16:30:00,ARB/USDT,0.5183,0.5231,0.5166,0.5219,1.453538e+06
2024-10-03 17:00:00,ARB/USDT,0.5219,0.5291,0.5181,0.5279,1.788411e+06


In [11]:
rows_per_pair = 1441
pair_counts = df['Trading Pair'].value_counts()
pairs_to_drop = pair_counts[pair_counts < rows_per_pair].index
df = df[~df['Trading Pair'].isin(pairs_to_drop)].copy()

In [12]:
df['Trading Pair'].value_counts().unique()

array([1441])

In [15]:
trading_pairs = df['Trading Pair'].unique()

## Calculating 24-Hour trading volume

In [18]:
df['24-Hour Volume'] = df['Volume'].rolling(window=48).sum()

In [19]:
df['24-Hour Volume'] = df['24-Hour Volume'].fillna(0)
df

,Trading Pair,Open,High,Low,Close,Volume,24-Hour Volume
Timestamp,,,,,,,
2024-09-03 18:00:00,BTC/FDUSD,57787.8500,58034.6300,57665.9300,57968.2300,8.237867e+02,0.0
2024-09-03 18:30:00,BTC/FDUSD,57971.0700,58141.4900,57953.5300,58003.1500,6.733895e+02,0.0
2024-09-03 19:00:00,BTC/FDUSD,58000.3600,58350.0000,57978.8600,58099.4200,6.882739e+02,0.0
2024-09-03 19:30:00,BTC/FDUSD,58099.5300,58202.0000,57936.2000,58138.1900,8.848367e+02,0.0
2024-09-03 20:00:00,BTC/FDUSD,58141.8800,58342.0000,58133.8500,58241.0700,4.732882e+02,0.0
...,...,...,...,...,...,...,...
2024-10-03 16:00:00,ARB/USDT,0.5264,0.5274,0.5180,0.5183,1.847456e+06,76279558.7
2024-10-03 16:30:00,ARB/USDT,0.5183,0.5231,0.5166,0.5219,1.453538e+06,76186635.8
2024-10-03 17:00:00,ARB/USDT,0.5219,0.5291,0.5181,0.5279,1.788411e+06,76477410.3


## Fetching Order Book data

In [20]:
trading_pairs_cleaned = [pair.replace('/', '') for pair in trading_pairs]
for i in range(10):
    print(trading_pairs_cleaned[i])

BTCFDUSD
BTCUSDT
ETHFDUSD
ETHUSDT
USDCUSDT
SOLUSDT
SOLFDUSD
XRPUSDT
FDUSDUSDT
SUIUSDT


In [24]:
def fetch_order_book(pair):
    url = f'https://api.binance.com/api/v3/depth?symbol={pair}&limit=10'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        time.sleep(0.5)
        return response.json()
    
    except requests.exceptions.HTTPError as e:
        if response.status_code == 429:
            print(f"Error {response.status_code}: Too many requests. Waiting for 60 seconds...")
            time.sleep(60)
            return fetch_order_book(pair)
        else:
            print(f"HTTP error occurred: {e}")
            return {'bids': [], 'asks': []}

In [25]:
def collect_order_books(cleaned_trading_pairs, start_time, end_time, interval):
    all_data = []
    times = [start_time + i * interval for i in range((end_time - start_time) // interval + 1)]
    total_requests = len(trading_pairs) * len(times)

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        for pair in cleaned_trading_pairs:
            for timestamp in times:
                futures.append(executor.submit(fetch_order_book, pair))

        for future in tqdm(concurrent.futures.as_completed(futures), total=total_requests, unit=' requests', desc='Fetching Order Book Data'):
            order_book = future.result()
            pair_index = futures.index(future) // len(times)
            timestamp_index = futures.index(future) % len(times)
            pair = trading_pairs[pair_index]
            timestamp = times[timestamp_index].strftime('%Y-%m-%d %H:%M:%S')

            all_data.append({
                'Timestamp': timestamp,
                'Trading Pair': pair,
                'Bids': order_book['bids'],
                'Asks': order_book['asks'],
            })

    return all_data

In [26]:
start_time = datetime(2024, 9,  3, 18, 00, 0)
end_time = datetime(2024, 10, 3, 18, 00, 0)
interval = timedelta(minutes=30)

In [27]:
data = collect_order_books(trading_pairs_cleaned, start_time, end_time, interval)
order_book_df = pd.DataFrame(data)  

Fetching Order Book Data:   0%|          | 0/66286 [00:00<?, ? requests/s]

In [28]:
expanded_data = []
for index, row in order_book_df.iterrows():
    timestamp = row['Timestamp']
    pair = row['Trading Pair']      
    bids = row['Bids']
    asks = row['Asks']
    max_length = len(bids)
    
    for i in range(max_length):
        bid_price = bids[i][0]
        bid_quantity = bids[i][1]
        ask_price = asks[i][0]
        ask_quantity = asks[i][1]
        
        expanded_data.append({
            'Timestamp': timestamp,
            'Trading Pair': pair,
            'Bid Price': bid_price,
            'Bid Quantity': bid_quantity,
            'Ask Price': ask_price,
            'Ask Quantity': ask_quantity,
        })

In [29]:
order_book_df = pd.DataFrame(expanded_data)
order_book_df['Timestamp'] = pd.to_datetime(order_book_df['Timestamp'])
order_book_df = order_book_df.set_index('Timestamp')
order_book_df.head()

,Trading Pair,Bid Price,Bid Quantity,Ask Price,Ask Quantity
Timestamp,,,,,
2024-09-03 18:30:00,BTC/FDUSD,60630.00000000,0.00300000,60631.84000000,0.03770000
2024-09-03 18:30:00,BTC/FDUSD,60628.79000000,0.03800000,60633.73000000,0.09045000
2024-09-03 18:30:00,BTC/FDUSD,60628.66000000,0.03800000,60633.74000000,0.03800000
2024-09-03 18:30:00,BTC/FDUSD,60628.53000000,0.03800000,60634.00000000,0.13180000
2024-09-03 18:30:00,BTC/FDUSD,60628.39000000,0.03800000,60635.07000000,0.00630000


In [30]:
order_book_df

,Trading Pair,Bid Price,Bid Quantity,Ask Price,Ask Quantity
Timestamp,,,,,
2024-09-03 18:30:00,BTC/FDUSD,60630.00000000,0.00300000,60631.84000000,0.03770000
2024-09-03 18:30:00,BTC/FDUSD,60628.79000000,0.03800000,60633.73000000,0.09045000
2024-09-03 18:30:00,BTC/FDUSD,60628.66000000,0.03800000,60633.74000000,0.03800000
2024-09-03 18:30:00,BTC/FDUSD,60628.53000000,0.03800000,60634.00000000,0.13180000
2024-09-03 18:30:00,BTC/FDUSD,60628.39000000,0.03800000,60635.07000000,0.00630000
...,...,...,...,...,...
2024-10-03 18:00:00,ARB/USDT,0.53150000,27429.40000000,0.53260000,51531.00000000
2024-10-03 18:00:00,ARB/USDT,0.53140000,42721.60000000,0.53270000,30513.50000000
2024-10-03 18:00:00,ARB/USDT,0.53130000,119199.30000000,0.53280000,16408.00000000


## Saving to Disk

In [31]:
df.to_csv('OHLCV_Trading_Volume.csv')
order_book_df.to_csv('Order_Book.csv')

## Notes

- Some trading pairs, being new, have lesser OHLCV data.

- Used 24h summed up trading volume, where first day might have 0 as trading volume